# Start_RAG_with_Gemini_and_Langchain
제작 : 박광석(모두의연구소, https://www.linkedin.com/in/andkspark)   
  
해당 노트는 Langchain으로 RAG를 구현하기 위해 필요한  
각 컴포넌트인 Document Loaders, Text splitters, Text embeddings, Vectorstores, Retriever를 다룹니다    
참고 :
https://python.langchain.com/docs/get_started/introduction  
https://python.langchain.com/docs/integrations/llms/google_ai/  
YouTube <모두의AI> , <테디노트>   
인프런 <모두를 위한 대규모 언어 모델 LLM(Large Language Model) Part 2 - 랭체인(LangChain)으로 나만의 ChatGPT 만들기>



### Step 0 : 설치와 준비  
Langchain 설치 및 Gemini API 키를 등록하도록 합니다.  

In [ ]:
#! curl ipinfo.io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
YOUR_API_KEY = ''

In [ ]:
!pip install U -q langchain langchain-google-genai
!pip install U -q langchain-community langchain-core


In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

In [ ]:
#랭체인의 구글Api를 사용합니다
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature=0.0)

In [ ]:
!pip install -q pypdf pdf2image docx2txt pdfminer unstructured #의존성 모듈을 설치합니다

### Step 1 : Document Loaders 사용해보기  
Document Loader는 PDF, 웹페이지, CSV 등 여러 종류의 문서를 text로 파싱하는 기능을 가지고 있습니다.  
https://python.langchain.com/docs/modules/data_connection/document_loaders/  

https://it.chosun.com/news/articleView.html?idxno=2023092111831

#### PDFLoader 사용  
읽고 싶은 pdf 파일을 드라이브에 올려주세요!    

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Demian.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

Document(metadata={'producer': 'Adobe Acrobat Standard DC 19 Paper Capture Plug-in', 'creator': 'ScanFix(TM) Enhanced', 'creationdate': '2015-09-10T01:40:29+00:00', 'moddate': '2019-01-30T17:47:47+01:00', 'source': '/content/Demian.pdf', 'total_pages': 182, 'page': 0, 'page_label': '1'}, page_content='DEMIAN \n• \nDownloaded from https://www.holybooks.com')

In [ ]:
print(pages[10])

page_content='TWO WOR.LDS 
Finally, out of sheer nervousness, I began to talk. I 
invented a long story of robbery, in which I featured as 
the hero. One night in the comer by the mill a friend 
and I ha.d stolen a whole sackful of apples, not just 
ordinary apples but pippins, golden pippins of the best 
kind at that. I was taking refuge in my story from the 
dangers of the moment and found no difficulty in invent­
ing and relating it. In order not to dry up too soon and 
perhaps become involved in something worse, I gave full 
rein to my narrative powers. One of us, I reported, had 
always stood guard while the other sat in the tree and 
chucked the apples down, and the sack had got so heavy 
that in the end we had to open it and leave half behind, 
but we came back half an hour later and fetched them 
too. 
I hoped for some applause at the end of my story; I 
had warmed up to the narrative aJ: last, carried away by 
my own eloquence. The two smaller boys were silent, 
waiting, Lut F

깔끔한 출력을 위해, page_content키만 접근해보겠습니다.

In [ ]:
print(pages[10].page_content)

TWO WOR.LDS 
Finally, out of sheer nervousness, I began to talk. I 
invented a long story of robbery, in which I featured as 
the hero. One night in the comer by the mill a friend 
and I ha.d stolen a whole sackful of apples, not just 
ordinary apples but pippins, golden pippins of the best 
kind at that. I was taking refuge in my story from the 
dangers of the moment and found no difficulty in invent­
ing and relating it. In order not to dry up too soon and 
perhaps become involved in something worse, I gave full 
rein to my narrative powers. One of us, I reported, had 
always stood guard while the other sat in the tree and 
chucked the apples down, and the sack had got so heavy 
that in the end we had to open it and leave half behind, 
but we came back half an hour later and fetched them 
too. 
I hoped for some applause at the end of my story; I 
had warmed up to the narrative aJ: last, carried away by 
my own eloquence. The two smaller boys were silent, 
waiting, Lut Franz Kromer ga

#### CSVLoader

읽고 싶은 CSV 파일을 드라이브에 올려주세요!    

In [ ]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader("/content/titanic.csv")

data = loader.load()

In [ ]:
data[:3]

[Document(metadata={'source': '/content/titanic.csv', 'row': 0}, page_content='PassengerId: 1\nSurvived: 0\nPclass: 3\nName: Braund, Mr. Owen Harris\nSex: male\nAge: 22\nSibSp: 1\nParch: 0\nTicket: A/5 21171\nFare: 7.25\nCabin: \nEmbarked: S'),
 Document(metadata={'source': '/content/titanic.csv', 'row': 1}, page_content='PassengerId: 2\nSurvived: 1\nPclass: 1\nName: Cumings, Mrs. John Bradley (Florence Briggs Thayer)\nSex: female\nAge: 38\nSibSp: 1\nParch: 0\nTicket: PC 17599\nFare: 71.2833\nCabin: C85\nEmbarked: C'),
 Document(metadata={'source': '/content/titanic.csv', 'row': 2}, page_content='PassengerId: 3\nSurvived: 1\nPclass: 3\nName: Heikkinen, Miss. Laina\nSex: female\nAge: 26\nSibSp: 0\nParch: 0\nTicket: STON/O2. 3101282\nFare: 7.925\nCabin: \nEmbarked: S')]

#### 웹베이스로더  
웹페이지를 파싱합니다

In [ ]:
from langchain.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://it.chosun.com/news/articleView.html?idxno=2023092111831")
documents = loader.load()

#print(documents[0].page_content)

주석을 풀고 수행하면, 해당 페이지의 모든 텍스트를 읽어옵니다.  
    
모든 형식을 텍스트로 잘 읽어왔습니다.  
전처리 후 잘 사용할 수 있을 것 같습니다!

### Step2 : TextSplitters 사용해보기  
TextSplitter는 지정한 구분자, 혹은 다른 특정 기준을 특징으로 텍스트를 Chunk로 나누는 역할을 수행합니다.    
LLM은 Splitters를 통해 분할된 여러 문서들을 입력받을 수 있으며, 이를 통해 토큰 길이의 제약을 극복해냅니다.  
각각의 chunk들은 vectorstore에 1:1로 임베딩됩니다

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

CharacterTextSplitter는 구분자 1개를 기준으로 문장을 분할합니다  
max_token을 초과하는 경우가 생깁니다.  
  
RecursiveCharacterTextSplitter는 줄바꿈, 마침표, 쉼표 등 구분자 여러개를 돌아가며 재귀적으로 분할합니다.  
완전하지 않은 문장으로 분할하는 경우가 생깁니다.  

In [ ]:
with open("/content/state_of_the_union.txt") as f:
    text = f.read()

In [ ]:
#len은 어떤 기준으로 chunk size를 잴 것인가?의 기준이 되는 함수입니다.
#chunk_overlap은 chunk의 앞뒤로 다른 chunk와 설정한 size까지 겹칠 수 있도록 설정하는 것입니다.
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=100, length_function = len,)
chunks = text_splitter.split_text(text)

Chunk의 내용을 확인해보겠습니다

In [ ]:
print(chunks[0])

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.


각 chunk의 길이를 확인해보겠습니다,

In [ ]:
length = []
for chunk in chunks:
    length.append(len(chunk))

print(length)

[939, 995, 810, 962, 994, 883, 957, 952, 928, 915, 993, 702, 900, 950, 957, 958, 967, 996, 796, 866, 888, 966, 964, 977, 998, 948, 925, 924, 989, 965, 938, 936, 981, 965, 771, 982, 972, 977, 984, 999, 968, 801]


### 토큰 단위로 텍스트 분할해보기  
  
LLM의 경우 토큰을 기준으로 자연어를 처리하기 때문에, 자연어의 form인 단어 길이는 서비스 개발에서 적합하지 않을 수 있습니다.  
토큰 단위로 텍스트를 분할해보도록 하겠습니다!  


In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text
    )
    return len(tokens)

In [ ]:
tiktoken_length = []
for chunk in chunks:
    tiktoken_length.append(tiktoken_len(chunk))

print(length)
print(tiktoken_length)

[939, 995, 810, 962, 994, 883, 957, 952, 928, 915, 993, 702, 900, 950, 957, 958, 967, 996, 796, 866, 888, 966, 964, 977, 998, 948, 925, 924, 989, 965, 938, 936, 981, 965, 771, 982, 972, 977, 984, 999, 968, 801]
[197, 198, 163, 190, 203, 182, 195, 197, 206, 205, 218, 148, 188, 205, 216, 215, 209, 224, 176, 187, 201, 197, 201, 215, 222, 202, 203, 204, 229, 206, 184, 204, 197, 194, 156, 200, 194, 221, 203, 225, 209, 187]


글자 수와 토큰 수의 차이를 확인할 수 있습니다 !

### Step3 : TextEmbedding 사용해보기  
컴퓨터가 텍스트를 이해할 수 있도록 벡터로 변환합니다.  
변환된 벡터는 벡터스토어에 저장되거나, 기존의 다른 벡터와의 유사성을 계산할 수 있습니다.  
변환은 대용량의 말뭉치로 사전훈련된 임베딩 모델을 통해 이루어지게 됩니다.  
  
Gemini pro의 임베딩 모델을 사용해보겠습니다!  
https://ai.google.dev/docs/embeddings_guide?hl=ko

In [ ]:
import google.generativeai as genai

genai 라이브러리의 list_models 함수를 사용하여 사용 가능한 모델들의 목록을 가져옵니다.

In [ ]:
genai.configure(api_key=YOUR_API_KEY)

In [ ]:
for model in genai.list_models():
    if "embedContent" in model.supported_generation_methods:
        print(model.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


두 개의 임베딩 모델을 사용할 수 있습니다  

https://github.com/google/generative-ai-docs/blob/main/examples/gemini/python/langchain/Gemini_LangChain_QA_Chroma_WebLoad.ipynb

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

Gemini embedding은 지역에 따라 사용이 제한됩니다.  
주로 유럽권에서 제한되기 때문에, 다음 에러를 확인하신다면 Colab 파일의 서버 저장 위치를 확인 후, 다른 임베딩 모델로 변경해야합니다.  

Error embedding content: 400 User location is not supported for the API use.


In [ ]:
#!curl ipinfo.io

In [ ]:
# 400 User location is not supported for the API use 오류가 발생한다면, 이 블록을 대신 실행해주세요

# ! pip install -q sentence_transformers

#from langchain.embeddings import HuggingFaceEmbeddings
#embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding model 변수에 google의 임베딩모델 혹은 huggingface의 임베딩모델이 할당되었을 것입니다.  
embed_documents 멤버 함수를 사용하여 새 문장을 변환해보겠습니다  

In [ ]:
embeddings = embedding_model.embed_documents(
    [
        "This is red apple.",
        "This is yellow banana.",
        "This is green lime.",
    ]
)

임베딩으로 잘 변환되었는지 확인해보겠습니다  

In [ ]:
print(embeddings[1])

[-0.016296545043587685, 0.01907380484044552, -0.00865683238953352, -0.03434057533740997, 0.012856596149504185, 0.01864556409418583, -0.016674622893333435, -0.007018635515123606, 0.021180110052227974, 0.04353297874331474, -0.0013039407785981894, 0.05327136814594269, 0.035700876265764236, -0.0037640670780092478, -0.013039857149124146, -0.02502686157822609, 0.046053025871515274, 0.08711031824350357, -0.09746488183736801, -0.002442749449983239, 0.07618019729852676, -0.015109164640307426, 0.08231203258037567, 0.022819723933935165, -0.02144741825759411, -0.0007384165655821562, -0.055751338601112366, 0.019622115418314934, 0.009542467072606087, -0.024235006421804428, 0.05243077501654625, 0.0661461129784584, 0.02837509661912918, -0.014317168854176998, 0.0016176634235307574, 0.03898113965988159, -0.01096457801759243, 0.027868635952472687, 0.042691804468631744, -0.053747668862342834, -0.07162857055664062, 0.0499441921710968, 0.002752243308350444, 0.026926163583993912, -0.03703746572136879, -0.015

In [ ]:
len(embeddings[1])

768

새로운 쿼리를 넣어, 임베딩끼리 유사도를 계산해보겠습니다

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
query = ["this is red fruit"]

In [ ]:
e_query = embedding_model.embed_documents(query)
print(cos_sim(embeddings[0], e_query[0]))
print(cos_sim(embeddings[1], e_query[0]))
print(cos_sim(embeddings[2], e_query[0]))

0.9281472104438462
0.8610421272571303
0.8125816090062714


빨간 사과와 빨간 과일의 유사도가 많이 높게 나왔습니다!  
  
임베딩 모델은 사용 언어나 필요에 따라 다양하게 교체하여 사용할 수 있습니다.  
해당 링크에서 여러 목록을 확인하실 수 있습니다.  
https://python.langchain.com/docs/integrations/text_embedding/

### Step4 : VectorStore 사용해보기
VectorStore는 자연어를 Vector로 embedding한 후, 이를 저장하는 공간입니다.  
쿼리나 문서 등의 Emabedding이 저장된 후, 재사용 및 참조를 위해 VectorStroe에서 빠르게 유사도를 측정, 탐색할 수 있습니다.  
VectorStore로는 Chroma나 Faiss를 가장 많이 사용합니다.  
  
예제로는, 가장 간단한 chromadb를 사용해보겠습니다.  

In [ ]:
!pip install chromadb

In [ ]:
#!pip install langchain-chroma

In [ ]:
#from langchain_community.vectorstores import Chroma

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
#!pip install --upgrade opentelemetry-api
#!pip install --upgrade opentelemetry-sdk

In [ ]:
#from langchain_chroma import Chroma

제일 처음에 사용했던, PDF를 다시 사용하도록 합니다!  

In [ ]:
# 위에서 사용했던 코드입니다
loader = PyPDFLoader("/content/Demian.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function = tiktoken_len)
docs = text_splitter.split_documents(pages)

In [ ]:
#!pip show chromadb

Chroma에 임베딩 시킵니다  

In [ ]:
db = Chroma.from_documents(docs, embedding_model)


이제 쿼리를 날려보겠습니다

In [ ]:
query = "how Demian look like?"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

DEMIAN 
with a feeling of nausea, I noticed Demian's expression. 
He had not thrust himself to the front but stood right 
at the back, looking elc!gant and at ease as usual. His 
glance seemed directed at the horse's head, and again it 
. showed that deep, quiet, almost fanatical yet passionate 
absorption. I could not help staring at him for some 
moments and it was then that I felt aware of a very 
uncanny sensation in my remote consciousness. I saw 
Demian's face and remarked that it was not a boy's face 
but a man's and then I saw, or rather became aware, that 
it was not really the face of a man either; it had some­
thing different about it, almost a feminine element. And 
for the time being his face seemed neither masculine 
nor childish, neither old nor young but a hundred years 
old, almost timeless and bearing the mark of other 
periods of history than our own. Animals might look 
thus, trees or stars. I did not know then, of course, I 
did not feel exactly what I am writing a

Face, features, looks like 등 데미안의 생김새를 담고 있는 페이지가 출력되었습니다  
굉장히 빠른 속도로 검색했습니다!  

### Step5 : Retriever 사용해보기  

Retriever는 사용자 입력 시 임베딩 모델을 거친 쿼리와 vectorstore에 저장된 사전 정보들 간의 유사 문장을 찾아내어 출력하는 역할을 합니다.  
  




In [ ]:
from langchain.chains import RetrievalQA


긴 문서에서 원하는 답변을 찾을 수 있는 RetrieverQA 체인을 사용합니다  

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature=0.0)

체인의 종류, 탐색 방법과 그에 따른 파라메터를 설정해줍니다  

In [ ]:
qa = RetrievalQA.from_chain_type(llm, chain_type="stuff",
                                 retriever=db.as_retriever(
                                     search_type="mmr",
                                     search_kwargs={"k": 3, "fetch_k" : 10}),
                                 return_source_documents=True)

In [ ]:
query = "how demian looks like"
result = qa(query)

마크다운 형식으로 출력해봅니다

In [ ]:
from IPython.display import Markdown, display
display(Markdown(result["result"]))

The descriptions of Demian's appearance are varied and somewhat contradictory, suggesting a mysterious and perhaps even otherworldly quality.  At times he is described as having a face that is neither boyish nor manly, neither old nor young, but timeless and almost animalistic.  He is also described as handsome, yet cold and stone-like, with an aura of quiet emptiness.  In one instance, the narrator notes a "feminine element" to his face.  Later, Demian is seen wearing a fawn mackintosh and carrying a walking stick.  The overall impression is one of someone unusual and captivating, defying easy categorization.

RAG를 사용하지 않은 llm 호출도 시도해보세요!

In [ ]:

llm2 = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature=0.0)
request = llm2.invoke("how demian looks like")
display(Markdown(request.content))


Descriptions of Demian's appearance in Hermann Hesse's novel *Demian* are sparse and somewhat ambiguous, reflecting the novel's focus on inner transformation rather than physical details.  There's no single definitive image.  However, we can piece together some impressions:

* **Mysterious and somewhat unsettling:**  He's presented as possessing an air of mystery and a certain darkness that intrigues the young Sinclair.  This isn't necessarily a physical attribute, but rather an aura or presence.

* **Older and more mature than Sinclair:**  Demian is significantly older than Sinclair, giving him a perceived maturity and worldly wisdom that draws Sinclair to him.  This would suggest a more developed physique and perhaps a more serious demeanor.

* **Not conventionally handsome:**  There's no suggestion that Demian is conventionally handsome.  His appeal lies in his personality and his intellectual and spiritual depth, not his physical attractiveness.

* **Possibly androgynous:** Some interpretations suggest a hint of androgynous features, reflecting his complex and multifaceted nature. This is more a symbolic interpretation than a literal description.

In short, Demian's appearance is left largely to the reader's imagination. He's not described in detail because his significance lies in his influence on Sinclair's spiritual and psychological development, not his physical form.  Any visual representation would be an interpretation of the novel's themes and the reader's own perception of the character.

수고하셨습니다!